In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/25 15:31:34 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/09/25 15:31:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 15:31:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
orders = spark.read.parquet("../../../data/insights/pre_insights/orders.parquet/")

In [4]:
orders.dtypes

[('merchant_abn', 'bigint'),
 ('merchant_name', 'string'),
 ('consumer_id', 'bigint'),
 ('order_datetime', 'date'),
 ('dollar_value', 'double')]

In [5]:
orders.show(truncate= False)

+------------+------------------------------------+-----------+--------------+------------+
|merchant_abn|merchant_name                       |consumer_id|order_datetime|dollar_value|
+------------+------------------------------------+-----------+--------------+------------+
|92980848589 |Maecenas Industries                 |564558     |2022-08-30    |479.0       |
|58454491168 |Diam At Foundation                  |564558     |2022-08-24    |69.57       |
|24852446429 |Erat Vitae LLP                      |564558     |2022-04-18    |12.87       |
|45559085309 |A Auctor Non Corporation            |564558     |2021-07-07    |48.19       |
|58392414752 |Mattis Ornare Lectus Inc.           |564558     |2022-09-26    |2.76        |
|81219314324 |Faucibus Leo Ltd                    |564558     |2022-10-08    |120.55      |
|34179569263 |Sodales At LLC                      |564558     |2021-12-05    |816.97      |
|96566672398 |Odio Tristique Pharetra Incorporated|564558     |2021-04-15    |81

In [6]:
orders_aggregated = orders.groupBy("merchant_abn").agg\
                        (f.count("order_datetime").alias("number_of_orders"),
                         f.avg("dollar_value").alias("average_cost_of_order"))

In [7]:
orders_aggregated.show(truncate=False)

+------------+----------------+---------------------+
|merchant_abn|number_of_orders|average_cost_of_order|
+------------+----------------+---------------------+
|19839532017 |614             |157.0                |
|83412691377 |11928           |34.97127431254191    |
|38700038932 |5944            |1344.3426917900406   |
|15613631617 |1483            |303.77948078219816   |
|73256306726 |4361            |283.94453107085536   |
|73841664453 |811             |85.53958076448828    |
|60654402457 |158             |85.27993670886076    |
|96946925998 |110             |1004.9574545454543   |
|35344855546 |1274            |89.12368131868134    |
|52763133264 |111             |116.72171171171172   |
|12516851436 |179             |155.86396648044695   |
|48214071373 |453             |303.36114790286973   |
|41956465747 |244             |225.88536885245904   |
|92202115241 |98              |330.3822448979592    |
|34440496342 |187             |89.50550802139038    |
|78916025936 |52            

In [8]:
orders_aggregated.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/orders_agg.parquet")

In [9]:
spark.stop()